# UChicago MSCA 32009 Health Analytics Project

[Kaggle Competition](https://www.kaggle.com/c/pf2012-diabetes/): Practice Fusion Diabetes Classification

Elly (Linqi) Yang

In [1]:
import numpy as np
import pandas as pd

## select features to be included in model

In [2]:
def select(df):
    
    # SyncTranscript data
    Transcript = pd.read_csv('training_SyncTranscript.csv')
    df = df.merge(Transcript.groupby(by = 'PatientGuid').size().to_frame('nTranscript'), on = 'PatientGuid', how = 'left')

    TranscriptList = ['Height', 'Weight', 'BMI', 'SystolicBP', 'DiastolicBP', 'RespiratoryRate']
    for i in TranscriptList: Transcript.loc[Transcript[i].isnull(), i] = Transcript.loc[~Transcript[i].isnull(), i].mean()
    df = df.merge(Transcript.groupby(by = 'PatientGuid')[TranscriptList].mean(), on = 'PatientGuid', how = 'left')

    TranscriptPS = list(Transcript['PhysicianSpecialty'].value_counts()[:5].index)
    for i in TranscriptPS:
        n = Transcript[Transcript['PhysicianSpecialty'] == i].groupby(by = 'PatientGuid').size()
        df = df.merge(n.to_frame('nTranscriptPS'+str(TranscriptPS.index(i))), on = 'PatientGuid', how = 'left')
    
    # SyncSmokingStatus data & SyncPatientSmokingStatus data
    SmokingStatus = pd.read_csv('SyncSmokingStatus.csv')
    SmokingStatus['nCig'] = 0
    nCigs = ['Few (1-3) cigarettes per day', 'Up to 1 pack per day', '1-2 packs per day', '2 or more packs per day']
    for i in nCigs: SmokingStatus.loc[SmokingStatus['Description'] == i, 'nCig'] = nCigs.index(i)+1

    PatientSmokingStatus = pd.read_csv('training_SyncPatientSmokingStatus.csv')
    PatientSmokingStatus = PatientSmokingStatus.merge(SmokingStatus, on = 'SmokingStatusGuid', how = 'left')
    df = df.merge(PatientSmokingStatus.groupby(by = 'PatientGuid').mean()['nCig'].to_frame('avgCig'), on = 'PatientGuid', how = 'left')
    
    # SyncLabResult data
    LabResult = pd.read_csv('training_SyncLabResult.csv')
    df = df.merge(LabResult.groupby(by = 'PatientGuid').size().to_frame('nLabResult'), on = 'PatientGuid', how = 'left')
    
    # SyncLabPanel data
    LabPanel = pd.read_csv('training_SyncLabPanel.csv')
    LabPanel = LabPanel.merge(LabResult[['LabResultGuid', 'PatientGuid']])
    df = df.merge(LabPanel.groupby(by = 'PatientGuid').size().to_frame('nLabPanel'), on = 'PatientGuid', how = 'left')

    LabPanelPN = list(LabPanel['PanelName'].value_counts()[:5].index)
    for i in LabPanelPN:
        n = LabPanel[LabPanel['PanelName'] == i].groupby(by = 'PatientGuid').size()
        df = df.merge(n.to_frame('nLabPanelPN'+str(LabPanelPN.index(i))), on = 'PatientGuid', how = 'left')

    # SyncLabObservation data
    LabObs = pd.read_csv('training_SyncLabObservation.csv')
    LabObs = LabObs.merge(LabPanel[['LabPanelGuid', 'PatientGuid']])
    df = df.merge(LabObs.groupby(by = 'PatientGuid').size().to_frame('nLabObs'), on = 'PatientGuid', how = 'left')

    LabObsHL = list(LabObs['HL7Text'].value_counts()[:5].index)
    for i in LabObsHL:
        n = LabObs[LabObs['HL7Text'] == i].groupby(by = 'PatientGuid').size()
        df = df.merge(n.to_frame('nLabObsHL'+str(LabObsHL.index(i))), on = 'PatientGuid', how = 'left')
    
    # SyncDiagnosis data
    Diagnosis = pd.read_csv('training_SyncDiagnosis.csv')
    df = df.merge(Diagnosis.groupby(by = 'PatientGuid').size().to_frame('nDiagnosis'), on = 'PatientGuid', how = 'left')
    df = df.merge(Diagnosis.groupby(by = 'PatientGuid').sum()['Acute'].to_frame('nAcute'), on = 'PatientGuid', how = 'left')

    Diagnosis['ICD9Num'] = Diagnosis.loc[Diagnosis['ICD9Code'].str[0].str.isdigit(), 'ICD9Code'].astype(float)
    Diagnosis['ICD9Group'] = pd.cut(Diagnosis['ICD9Num'], 
                                    [1, 140, 240, 280, 290, 320, 360, 390, 460, 520, 580, 630, 680, 710, 740, 760, 780, 800, 1000], 
                                    labels = ['icd9_' + str(i+1) for i in range(18)], 
                                    include_lowest = True)
    Diagnosis['ICD9Group'] = Diagnosis['ICD9Group'].astype(str)
    Diagnosis.loc[Diagnosis['ICD9Group']=='nan', 'ICD9Group'] = 'icd9_19'

    Diagnosis.loc[Diagnosis['ICD9Group'] == 'icd9_3', 'ICD9Group'] = 'icd9_3.0'
    Diagnosis['ICD9Group'] = pd.cut(Diagnosis['ICD9Num'],
                                    [240, 249, 260, 270, 280],
                                    labels = ['icd9_3.' + str(i+1) for i in range(4)], 
                                    include_lowest = True).astype(str).replace('nan', np.NaN).fillna(Diagnosis['ICD9Group'])

    Diagnosis.loc[Diagnosis['ICD9Group'] == 'icd9_8', 'ICD9Group'] = 'icd9_8.0'
    Diagnosis['ICD9Group'] = pd.cut(Diagnosis['ICD9Num'],
                                    [390, 393, 401, 410, 415, 420, 430, 440, 451, 460],
                                    labels = ['icd9_8.' + str(i+1) for i in range(9)], 
                                    include_lowest = True).astype(str).replace('nan', np.NaN).fillna(Diagnosis['ICD9Group'])

    for i in Diagnosis.loc[Diagnosis['ICD9Group'] == 'icd9_3.4', 'ICD9Num'].value_counts().head(5).index: 
        Diagnosis.loc[Diagnosis['ICD9Num'] == i, 'ICD9Group'] = 'icd9_3.4_' + str(i)

    for i in Diagnosis.loc[Diagnosis['ICD9Group'] == 'icd9_8.3', 'ICD9Num'].value_counts().head(2).index: 
        Diagnosis.loc[Diagnosis['ICD9Num'] == i, 'ICD9Group'] = 'icd9_8.3_' + str(i)

    df = df.merge(pd.crosstab(index = Diagnosis['PatientGuid'], columns = Diagnosis['ICD9Group']), on = 'PatientGuid', how = 'left')
    
    # SyncMedication data
    Medication = pd.read_csv('training_SyncMedication.csv')
    df = df.merge(Medication.groupby(by = 'PatientGuid').size().to_frame('nMedication'), on = 'PatientGuid', how = 'left')

    MedicationMN = list(Medication['MedicationName'].value_counts()[:5].index)
    for i in MedicationMN:
        n = Medication[Medication['MedicationName'] == i].groupby(by = 'PatientGuid').size()
        df = df.merge(n.to_frame('nMedicationMN'+str(MedicationMN.index(i))), on = 'PatientGuid', how = 'left')

    Med_Diag = pd.merge(Medication, Diagnosis, on = ['PatientGuid', 'DiagnosisGuid'])
    Med_Diag_ct = pd.crosstab(index = Med_Diag['PatientGuid'], columns = Med_Diag['ICD9Group'])
    Med_Diag_ct.columns = ['med_'+i for i in Med_Diag_ct.columns]
    df = df.merge(Med_Diag_ct, on = 'PatientGuid', how = 'left')
    
    return(df)

## load train data

In [3]:
train = pd.read_csv('training_SyncPatient.csv', nrows = 6600)
train = select(train)
train_X = pd.get_dummies(train.drop(['PatientGuid', 'DMIndicator', 'PracticeGuid', 'State'], axis=1).fillna(0), drop_first = True)
train_y = train['DMIndicator']

## build random forest model

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
np.random.seed(123)
rf = RandomForestClassifier(n_estimators=4000, max_features=20, min_samples_leaf=20, oob_score=True)
rf.fit(train_X, train_y)

RandomForestClassifier(max_features=20, min_samples_leaf=20, n_estimators=4000,
                       oob_score=True)

## fit model on test data and calculate Brier Score

In [5]:
test = pd.read_csv('test_SyncPatient.csv')
test = select(test.iloc[6600:9948])
test_X = pd.get_dummies(test.drop(['PatientGuid', 'PracticeGuid', 'State'], axis=1).fillna(0), drop_first = True)
test_y = pd.read_csv('test_SyncPatient2.csv').iloc[6600:9948]['DMIndicator']
sum((rf.predict_proba(test_X)[:, 1] - test_y)**2)/test_y.shape[0]

0.1203230817263394

## reference
* https://en.wikipedia.org/wiki/List_of_ICD-9_codes
* https://en.wikipedia.org/wiki/Brier_score